In [14]:
import os
import numpy as np
from keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf
import keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# WITH PREPROCESSING

In [6]:
ds_train2 = image_dataset_from_directory('../data/french_bird_db_25_50', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="training",
seed=42, batch_size=64)

ds_test2 = image_dataset_from_directory('../data/french_bird_db_25_50', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="validation",
seed=42, batch_size=64)

Found 1250 files belonging to 25 classes.
Using 1000 files for training.
Found 1250 files belonging to 25 classes.
Using 250 files for validation.


In [7]:
class_names = ds_train2.class_names
num_classes = len(ds_train2.class_names)

In [8]:
# preprocessing
preprocess_input = tf.keras.applications.resnet50.preprocess_input

ds_train2_preprocessed = ds_train2.map(lambda x, y: (preprocess_input(x), y))
ds_test2_preprocessed = ds_test2.map(lambda x, y: (preprocess_input(x), y))


In [18]:

# # Charger le modèle ResNet50 pré-entraîné avec les poids ImageNet
# base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# # Ajouter une couche de pooling global pour réduire la dimensionalité
# x = base_model.output
# x_2 = GlobalAveragePooling2D()(x)

# # x_3 = Dense(num_classes, activation='relu')(x_2)
# x_3 = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x_2)
# x_3 = Dropout(0.5)(x_3)

# # Ajouter une couche dense pour effectuer la classification
# output = Dense(num_classes, activation='softmax')(x_3)

# # Créer le modèle final en combinant le modèle de base et les couches supplémentaires
# model2 = Model(inputs=base_model.input, outputs=output)

# # Geler les couches du modèle de base pour éviter de les entraîner
# for layer in base_model.layers:
#     layer.trainable = False

# # Compiler le modèle
# model2.compile(optimizer=Adam(learning_rate=0.001), 
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#               metrics=[keras.metrics.SparseCategoricalAccuracy()])

# callbacks = [EarlyStopping(monitor='val_loss', patience=10)]


# # Entraîner le modèle sur votre dataset avec le callback personnalisé

# history = model2.fit(ds_train2_preprocessed, epochs=10, callbacks=callbacks, validation_data=ds_test2_preprocessed)

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - loss: 10.7165 - sparse_categorical_accuracy: 0.1475 - val_loss: 7.5446 - val_sparse_categorical_accuracy: 0.6760
Epoch 2/10
13/16 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 7.1604 - sparse_categorical_accuracy: 0.5933

KeyboardInterrupt: 

In [21]:
from tensorflow.keras.applications import EfficientNetB7

# Charger le modèle ResNet50 pré-entraîné avec les poids ImageNet
base_model = tf.keras.applications.EfficientNetB7(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Ajouter une couche de pooling global pour réduire la dimensionalité
x = base_model.output
x_2 = GlobalAveragePooling2D()(x)

# x_3 = Dense(num_classes, activation='relu')(x_2)
x_3 = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x_2)
x_3 = Dropout(0.5)(x_3)

# Ajouter une couche dense pour effectuer la classification
output = Dense(num_classes, activation='softmax')(x_3)

# Créer le modèle final en combinant le modèle de base et les couches supplémentaires
model2 = Model(inputs=base_model.input, outputs=output)

# Geler les couches du modèle de base pour éviter de les entraîner
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model2.compile(optimizer=Adam(learning_rate=0.001), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

callbacks = [EarlyStopping(monitor='val_loss', patience=10)]


# Entraîner le modèle sur votre dataset avec le callback personnalisé

history = model2.fit(ds_train2_preprocessed, epochs=10, callbacks=callbacks, validation_data=ds_test2_preprocessed)

258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 285s 14s/step - loss: 10.3972 - sparse_categorical_accuracy: 0.2828 - val_loss: 6.6425 - val_sparse_categorical_accuracy: 0.7200
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 137s 9s/step - loss: 6.0484 - sparse_categorical_accuracy: 0.7082 - val_loss: 4.2029 - val_sparse_categorical_accuracy: 0.7960
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 124s 8s/step - loss: 3.7967 - sparse_categorical_accuracy: 0.8219 - val_loss: 2.8435 - val_sparse_categorical_accuracy: 0.8080
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 120s 8s/step - loss: 2.5372 - sparse_categorical_accuracy: 0.8441 - val_loss: 2.0479 - val_sparse_categorical_accuracy: 0.8120
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 117s 7s/step - loss: 1.7876 - sparse_categorical_accuracy: 0.8669 - val_loss: 1.6640 - val_sparse_categorical_accuracy: 0.7840
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 116s 7s/step - loss: 1.4516 - sparse_categorical_accuracy: 0.8700 - val_loss: 1.3

In [10]:
for image_batch, labels_batch in ds_train2_preprocessed:
  batch_pred = model2.predict(image_batch)
  class_pred = np.argmax(batch_pred, axis=1)
  print(class_pred)
  print(labels_batch)
  break

2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 194ms/step
[ 2 18 14  5 22  5 11  5  6 16  2 17  3  0  6 11  0 11 21 16  6  4  1 18
 17 19 20  5 21 10 18  9 14 14 11  6  4 18 15  3 12 18 15  4 11 22 16  9
  5  7 13  1  6 18  6  8  6 22  6  7 10  7 20  3]
tf.Tensor(
[ 2 18 14  6 22  5 11  5  6 16  2 17  3  0  6 11  0 11 21 16  6  4  1 18
 17 19 20  5 21 10 18  9 14 14 11  6  4 18 15  3 12 18 15  4 11 22 16  9
  5  7 13  1  6 18  6  8  6 22  6  7 10  8 20  3], shape=(64,), dtype=int32)


In [11]:
for i, (image_batch, labels_batch) in enumerate(ds_test2_preprocessed):
  batch_pred = model2.predict(image_batch)
  class_pred = np.argmax(batch_pred, axis=1)
  
  print('batch No {} with result: '.format(i), sum((class_pred == labels_batch).numpy()) / len(class_pred) * 100, '%')

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 661ms/step
batch No 0 with result:  85.9375 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 658ms/step
batch No 1 with result:  84.375 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 656ms/step
batch No 2 with result:  89.0625 %
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step  
batch No 3 with result:  82.75862068965517 %


2024-07-03 16:53:12.729321: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
